# Contextual Compression for Enhanced RAG Systems

## Setting Up the Environment

In [1]:
import fitz
import numpy as np
import json
import re

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로

    Returns:
        str: PDF에서 추출된 전체 텍스트
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 전체 텍스트를 저장할 문자열 초기화

    # 각 페이지를 순회하며 텍스트 추출
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]               # 해당 페이지 가져오기
        text = page.get_text("text")         # 텍스트 형식으로 내용 추출
        all_text += text                     # 추출된 텍스트 누적

    # 추출된 전체 텍스트 반환
    return all_text

## Chunking the Extracted Text

In [4]:
def chunk_text(text, n=1000, overlap=200):
    """
    주어진 텍스트를 n자 단위로 분할하되, 각 청크 간에 overlap만큼 겹치게 합니다.

    Args:
        text (str): 분할할 텍스트
        n (int): 각 청크의 문자 수 (기본값: 1000)
        overlap (int): 청크 간 겹치는 문자 수 (기본값: 200)

    Returns:
        List[str]: 분할된 텍스트 청크 리스트
    """
    chunks = []  # 청크들을 저장할 리스트 초기화

    # n - overlap 만큼 이동하면서 청크 생성
    for i in range(0, len(text), n - overlap):
        # i에서 i + n까지의 텍스트 조각을 청크로 추가
        chunks.append(text[i:i + n])

    return chunks  # 생성된 청크 리스트 반환

## Building a Simple Vector Store

In [5]:
class SimpleVectorStore:
    """
    NumPy를 활용한 간단한 벡터 저장소 구현체입니다.
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []     # 임베딩 벡터 리스트
        self.texts = []       # 원본 텍스트 리스트
        self.metadata = []    # 각 텍스트의 메타데이터 리스트

    def add_item(self, text, embedding, metadata=None):
        """
        단일 텍스트 항목을 벡터 저장소에 추가합니다.

        Args:
            text (str): 원본 텍스트
            embedding (List[float]): 텍스트의 임베딩 벡터
            metadata (dict, optional): 추가 메타데이터 (기본값: 빈 딕셔너리)
        """
        self.vectors.append(np.array(embedding))  # 임베딩을 NumPy 배열로 변환하여 저장
        self.texts.append(text)                   # 원본 텍스트 저장
        self.metadata.append(metadata or {})      # 메타데이터 저장 (None일 경우 빈 딕셔너리)

    def similarity_search(self, query_embedding, k=5):
        """
        질의 임베딩과 가장 유사한 텍스트를 검색합니다.

        Args:
            query_embedding (List[float]): 질의 임베딩 벡터
            k (int): 반환할 상위 결과 개수

        Returns:
            List[Dict]: 유사한 항목 리스트 (텍스트, 메타데이터, 유사도 포함)
        """
        if not self.vectors:
            return []  # 저장된 벡터가 없으면 빈 리스트 반환

        # 질의 벡터를 NumPy 배열로 변환
        query_vector = np.array(query_embedding)

        # 각 벡터와의 코사인 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (
                np.linalg.norm(query_vector) * np.linalg.norm(vector)
            )
            similarities.append((i, similarity))  # 인덱스와 유사도 점수 저장

        # 유사도 기준 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 상위 k개 항목 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })

        return results

## Embedding Generation

In [6]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    주어진 텍스트에 대해 임베딩을 생성합니다.

    Args:
        text (str 또는 List[str]): 임베딩을 생성할 입력 텍스트(또는 텍스트 리스트)
        model (str): 사용할 임베딩 모델 이름

    Returns:
        List[float] 또는 List[List[float]]: 생성된 임베딩 벡터 또는 벡터 리스트
    """
    # 입력이 문자열 하나일 수도 있고, 문자열 리스트일 수도 있으므로 리스트 형태로 통일
    input_text = text if isinstance(text, list) else [text]

    # 지정된 모델을 사용하여 임베딩 생성 요청
    response = client.embeddings.create(
        model=model,
        input=input_text
    )

    # 입력이 단일 문자열이었을 경우, 첫 번째 임베딩만 반환
    if isinstance(text, str):
        return response.data[0].embedding

    # 여러 문자열일 경우, 모든 임베딩 리스트 반환
    return [item.embedding for item in response.data]

## Building Our Document Processing Pipeline

In [7]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    RAG 처리를 위한 문서 전처리 함수입니다.

    Args:
        pdf_path (str): PDF 파일 경로.
        chunk_size (int): 각 청크의 문자 수.
        chunk_overlap (int): 청크 간 겹치는 문자 수.

    Returns:
        SimpleVectorStore: 청크 및 임베딩이 저장된 벡터 저장소 객체.
    """
    print("PDF에서 텍스트를 추출합니다...")
    extracted_text = extract_text_from_pdf(pdf_path)

    print("텍스트를 청크 단위로 분할합니다...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"{len(chunks)}개의 청크가 생성되었습니다.")

    print("청크 임베딩을 생성합니다...")
    chunk_embeddings = create_embeddings(chunks)

    print("벡터 저장소를 초기화합니다...")
    store = SimpleVectorStore()

    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": pdf_path}
        )

    print(f"{len(chunks)}개의 청크가 벡터 저장소에 추가되었습니다.")
    return store

## Implementing Contextual Compression

In [8]:
def compress_chunk(chunk, query, compression_type="selective", model="gpt-4o-mini"):
    """
    주어진 텍스트 청크에서 질의에 관련된 정보만 추출하여 압축합니다.

    Args:
        chunk (str): 압축 대상 텍스트 청크
        query (str): 사용자 질의
        compression_type (str): 압축 방식 ("selective", "summary", "extraction")
        model (str): 사용할 LLM 모델 이름

    Returns:
        Tuple[str, float]: 압축된 청크 문자열과 압축률(%) 
    """
    
    # 압축 방식에 따라 시스템 프롬프트 설정
    if compression_type == "selective":
        # 관련 문장 또는 문단만 **선택적 추출**
        system_prompt = """귀하는 정보 필터링 전문가입니다.
        귀하의 임무는 문서 청크를 분석하여 사용자의 쿼리와 직접적으로 관련된 문장이나 단락만
        추출하는 것입니다. 관련 없는 콘텐츠는 모두 제거하세요.

        출력물은 다음과 같아야 합니다:
        1. 쿼리에 대한 답변에 도움이 되는 텍스트만 포함해야 합니다.
        2. 관련 문장의 정확한 표현을 유지하세요(의역하지 마세요).
        3. 텍스트의 원래 순서 유지
        4. 중복되는 것처럼 보이더라도 모든 관련 콘텐츠를 포함하세요.
        5. 쿼리와 관련이 없는 모든 텍스트 제외

        추가 설명 없이 일반 텍스트로 응답 형식을 지정하세요."""
    
    elif compression_type == "summary":
        # 관련 내용을 **요약** 형태로 제공
        system_prompt = """귀하는 요약의 전문가입니다.
        여러분의 임무는 제공된 청크의 간결한 요약을 작성하여 사용자의 쿼리와 관련된 정보에만 초점을 맞추는 것입니다.
        정보에만 초점을 맞춘 간결한 요약을 작성하는 것입니다.

        출력물은 다음과 같아야 합니다:
        1. 쿼리 관련 정보에 대해 간략하지만 포괄적으로 작성해야 합니다.
        2. 쿼리와 관련된 정보에만 집중해야 합니다.
        3. 관련 없는 세부 정보는 생략
        4. 중립적이고 사실적인 어조로 작성합니다.

        추가 설명 없이 일반 텍스트로 응답 형식을 지정합니다."""
    
    else:  # "extraction"
        # 질의 관련 **문장만 원문 그대로 추출**
        system_prompt = """귀하는 정보 추출 전문가입니다.
        귀하의 임무는 문서 청크에서 사용자의 쿼리에 대한 답변과 관련된 정보가 포함된 정확한 문장만
        정확한 문장만 추출하는 것입니다.

        출력은 다음과 같아야 합니다:
        1. 원본 텍스트에서 관련 문장의 직접 인용문만 포함해야 합니다.
        2. 원본 문구를 그대로 유지합니다(텍스트를 수정하지 않습니다).
        3. 쿼리와 직접 관련된 문장만 포함하세요.
        4. 추출된 문장을 개행으로 구분합니다.
        5. 주석이나 추가 텍스트를 추가하지 마세요.

        추가 설명 없이 일반 텍스트로 응답 형식을 지정합니다."""

    # 사용자 프롬프트 구성 (질의 + 문서 청크)
    user_prompt = f"""
        Query: {query}

        Document Chunk:
        {chunk}

        이 쿼리에 대한 답변과 관련된 콘텐츠만 추출합니다.
    """

    # LLM을 호출하여 압축된 응답 생성
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # 생성된 압축된 텍스트 추출
    compressed_chunk = response.choices[0].message.content.strip()

    # 압축률 계산 (압축 전과 후 길이 차이로 % 계산)
    original_length = len(chunk)
    compressed_length = len(compressed_chunk)
    compression_ratio = (original_length - compressed_length) / original_length * 100

    return compressed_chunk, compression_ratio

## Implementing Batch Compression

In [9]:
def batch_compress_chunks(chunks, query, compression_type="selective", model="gpt-4o-mini"):
    """
    여러 개의 텍스트 청크를 개별적으로 압축하여 반환합니다.

    Args:
        chunks (List[str]): 압축 대상이 되는 텍스트 청크 리스트.
        query (str): 사용자 질의.
        compression_type (str): 압축 방식 ("selective", "summary", "extraction").
        model (str): 사용할 LLM 모델 이름.

    Returns:
        List[Tuple[str, float]]: (압축된 텍스트, 개별 압축률)로 구성된 리스트.
    """
    print(f"{len(chunks)}개의 청크 압축을 시작합니다...")
    results = []
    total_original_length = 0
    total_compressed_length = 0

    for i, chunk in enumerate(chunks):
        print(f"[{i + 1}/{len(chunks)}] 청크 압축 중...")

        # 개별 청크 압축 수행
        compressed_chunk, compression_ratio = compress_chunk(
            chunk, query, compression_type, model
        )
        results.append((compressed_chunk, compression_ratio))

        total_original_length += len(chunk)
        total_compressed_length += len(compressed_chunk)

    # 전체 압축률 출력
    overall_ratio = (
        (total_original_length - total_compressed_length) / total_original_length * 100
        if total_original_length > 0 else 0.0
    )
    print(f"전체 압축률: {overall_ratio:.2f}%")

    return results

## Response Generation Function

In [10]:
def generate_response(query, context, model="gpt-4o-mini"):
    """
    질의(query)와 문맥(context)을 바탕으로 LLM 응답을 생성합니다.
    
    Args:
        query (str): 사용자 질의
        context (str): 압축된 청크에서 추출한 문맥 텍스트
        model (str): 사용할 LLM 모델 이름
        
    Returns:
        str: 생성된 응답 문자열
    """
    # 시스템 프롬프트 정의: LLM의 역할과 응답 조건 지정
    system_prompt = """당신은 유용한 AI 비서입니다. 제공된 문맥에만 근거하여 사용자의 질문에 답변하세요.
    문맥에서 답을 찾을 수 없는 경우 정보가 충분하지 않다고 말합니다."""

    # 사용자 프롬프트 구성: 문맥 + 질문
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        위의 문맥에만 근거하여 포괄적인 답변을 제공하세요.
    """

    # LLM 호출을 통해 응답 생성
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  # 응답 일관성을 위한 설정
    )

    # 생성된 응답 텍스트 반환
    return response.choices[0].message.content

## Complete RAG Pipeline with Contextual Compression

In [12]:
def rag_with_compression(pdf_path, query, k=10, compression_type="selective", model="gpt-4o-mini"):
    """
    압축 기반 문맥 생성이 포함된 RAG 파이프라인을 실행합니다.

    Args:
        pdf_path (str): PDF 문서 경로.
        query (str): 사용자 질의.
        k (int): 초기 검색할 청크 수.
        compression_type (str): 압축 방식 ("selective", "summary", "extraction").
        model (str): 사용할 LLM 모델 이름.

    Returns:
        dict: 쿼리, 압축된 문맥, 응답, 압축률 등의 결과 딕셔너리.
    """
    print("\n=== 문맥 압축 기반 RAG 실행 ===")
    print(f"질문: {query}")
    print(f"압축 방식: {compression_type}")

    # 1. 문서 전처리 → 벡터 저장소 생성
    vector_store = process_document(pdf_path)

    # 2. 쿼리 임베딩 생성
    query_embedding = create_embeddings(query)

    # 3. 유사도 기반 상위 k개 청크 검색
    print(f"상위 {k}개의 관련 청크 검색 중...")
    results = vector_store.similarity_search(query_embedding, k=k)
    retrieved_chunks = [result["text"] for result in results]

    # 4. 검색된 청크에 대해 압축 수행
    compressed_results = batch_compress_chunks(retrieved_chunks, query, compression_type, model)
    compressed_chunks = [result[0] for result in compressed_results]
    compression_ratios = [result[1] for result in compressed_results]

    # 5. 압축된 청크가 모두 비어 있는 경우 예외 처리
    filtered_chunks = [(chunk, ratio) for chunk, ratio in zip(compressed_chunks, compression_ratios) if chunk.strip()]

    if not filtered_chunks:
        print("⚠️ 모든 청크가 빈 문자열로 압축되었습니다. 원본 청크를 사용합니다.")
        filtered_chunks = [(chunk, 0.0) for chunk in retrieved_chunks]
    else:
        compressed_chunks, compression_ratios = zip(*filtered_chunks)

    # 6. 압축된 청크들을 문맥으로 구성
    context = "\n\n---\n\n".join(compressed_chunks)

    # 7. 문맥을 기반으로 응답 생성
    print("압축된 문맥을 기반으로 응답 생성 중...")
    response = generate_response(query, context, model)

    # 8. 결과 반환
    result = {
        "query": query,
        "original_chunks": retrieved_chunks,
        "compressed_chunks": compressed_chunks,
        "compression_ratios": compression_ratios,
        "context_length_reduction": f"{sum(compression_ratios)/len(compression_ratios):.2f}%",
        "response": response
    }

    print("\n=== 최종 응답 ===")
    print(response)

    return result

## Comparing RAG With and Without Compression

In [13]:
def standard_rag(pdf_path, query, k=10, model="gpt-4o-mini"):
    """
    LLM 기반 압축 없이 수행하는 표준 RAG 파이프라인입니다.

    Args:
        pdf_path (str): PDF 문서 경로.
        query (str): 사용자 질의.
        k (int): 검색할 상위 유사 청크 개수.
        model (str): 응답 생성을 위한 LLM 모델 이름.

    Returns:
        dict: 다음 정보를 포함한 결과 딕셔너리:
            - query: 사용자 질의
            - chunks: 검색된 청크 목록
            - response: LLM이 생성한 응답
    """
    print("\n***표준 RAG 실행***")
    print(f"질문: {query}")

    # 1. 문서 처리 및 벡터 저장소 생성
    vector_store = process_document(pdf_path)

    # 2. 쿼리 임베딩 생성
    query_embedding = create_embeddings(query)

    # 3. 상위 k개의 관련 청크 검색
    print(f"상위 {k}개의 청크 검색 중...")
    results = vector_store.similarity_search(query_embedding, k=k)
    retrieved_chunks = [result["text"] for result in results]

    # 4. 검색된 청크를 문맥 문자열로 조합
    context = "\n\n---\n\n".join(retrieved_chunks)

    # 5. 문맥 기반 LLM 응답 생성
    print("응답 생성 중...")
    response = generate_response(query, context, model)

    # 6. 결과 딕셔너리 구성 및 반환
    result = {
        "query": query,
        "chunks": retrieved_chunks,
        "response": response
    }

    print("\n***최종 응답***")
    print(response)

    return result

## Evaluating Our Approach

In [14]:
def evaluate_responses(query, responses, reference_answer):
    """
    여러 RAG 응답을 기준 정답과 비교하여 평가합니다.

    Args:
        query (str): 사용자 질의.
        responses (Dict[str, str]): 각 방식별 응답 딕셔너리. (예: {"standard": ..., "compressed": ...})
        reference_answer (str): 기준 정답.

    Returns:
        str: 평가 결과 텍스트.
    """
    # 시스템 프롬프트: 평가자 역할 정의
    system_prompt = """당신은 다양한 RAG 응답을 평가하는 공정한 평가자입니다.
    각 응답을 기준 정답과 비교하여 정확성, 포괄성, 관련성, 간결성을 기준으로 평가하고
    가장 우수한 응답부터 순위를 매기세요."""

    # 사용자 프롬프트 구성
    user_prompt = f"""
    Query: {query}

    [Reference Answer]
    {reference_answer}

    """

    # 각 방식별 응답 추가
    for method, response in responses.items():
        user_prompt += f"[{method.capitalize()} Response]\n{response}\n\n"

    # 평가 기준 안내 추가
    user_prompt += """
    각 응답을 다음 기준에 따라 평가하세요:
    1. 정확성 (기준 정답과의 사실 일치 여부)
    2. 포괄성 (질문에 대해 얼마나 완전하게 답했는지)
    3. 간결성 (불필요한 정보 없이 핵심만 전달했는지)
    4. 전반적인 품질

    각 응답에 대한 분석을 제공한 후, 가장 우수한 응답부터 순위를 정하고 그 이유를 설명하세요.
    """

    # 평가 요청 전송
    evaluation_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    return evaluation_response.choices[0].message.content

In [15]:
def evaluate_compression(pdf_path, query, reference_answer=None, compression_types=["selective", "summary", "extraction"]):
    """
    다양한 문맥 압축 방식과 standard RAG를 비교 평가합니다.

    Args:
        pdf_path (str): PDF 문서 경로.
        query (str): 사용자 질의.
        reference_answer (str): 기준 정답 (있을 경우 평가 수행).
        compression_types (List[str]): 평가 대상 압축 방식 목록.

    Returns:
        dict: 다음을 포함한 평가 결과 딕셔너리:
            - query: 질의
            - responses: 각 방식의 응답
            - evaluation: LLM 기반 평가 결과 텍스트
            - metrics: 압축률 및 문맥 길이 비교
            - standard_result: 압축 미사용 결과
            - compression_results: 압축 방식별 결과
    """
    print("\n*** 문맥 압축 방식 평가 시작 ***")
    print(f"질문: {query}")

    # 1. standard RAG 실행 (압축 없음)
    standard_result = standard_rag(pdf_path, query)

    # 2. 압축 방식별 RAG 결과 저장
    compression_results = {}
    for comp_type in compression_types:
        print(f"\n[{comp_type.upper()} 압축 방식 평가 중...]")
        compression_results[comp_type] = rag_with_compression(
            pdf_path=pdf_path,
            query=query,
            compression_type=comp_type
        )

    # 3. 방식별 응답 수집
    responses = {
        "standard": standard_result["response"]
    }
    for comp_type in compression_types:
        responses[comp_type] = compression_results[comp_type]["response"]

    # 4. 평가 수행 (참조 정답이 있는 경우)
    if reference_answer:
        evaluation = evaluate_responses(query, responses, reference_answer)
        print("\n*** 평가 결과 ***")
        print(evaluation)
    else:
        evaluation = "기준 정답이 제공되지 않아 자동 평가를 생략했습니다."

    # 5. 압축 방식별 메트릭 계산
    metrics = {}
    for comp_type in compression_types:
        avg_ratio = sum(compression_results[comp_type]["compression_ratios"]) / len(compression_results[comp_type]["compression_ratios"])
        metrics[comp_type] = {
            "avg_compression_ratio": f"{avg_ratio:.2f}%",
            "total_context_length": len("\n\n".join(compression_results[comp_type]["compressed_chunks"])),
            "original_context_length": len("\n\n".join(standard_result["chunks"]))
        }

    # 6. 결과 정리 및 반환
    return {
        "query": query,
        "responses": responses,
        "evaluation": evaluation,
        "metrics": metrics,
        "standard_result": standard_result,
        "compression_results": compression_results
    }

## Running Our Complete System (Custom Query)

In [16]:
# AI 윤리에 관한 정보를 담고 있는 PDF 문서 경로
pdf_path = "dataset/AI_Understanding.pdf" 

# 문서에서 관련 정보를 추출하기 위한 사용자 질의
query = "의사 결정에 AI를 사용하는 것과 관련된 윤리적 우려는 무엇인가요?"  

# (선택사항) 평가에 사용할 참조 정답
reference_answer = """
의사 결정에 AI를 사용하면 몇 가지 윤리적 문제가 제기됩니다.
- 특히 채용, 대출, 법 집행과 같은 중요한 영역에서 AI 모델의 편향성은 불공정하거나 차별적인 결과를 초래할 수 있습니다.
- AI 기반 의사 결정의 투명성과 설명 가능성이 부족하면 개인이 불공정한 결과에 이의를 제기하기 어렵습니다.
- AI 시스템이 명시적인 동의 없이 방대한 양의 개인 데이터를 처리하기 때문에 개인정보 보호 위험이 발생합니다.
- 자동화로 인한 일자리 대체 가능성은 사회적, 경제적 우려를 불러일으킵니다.
- 또한 AI 의사결정은 소수의 대형 기술 기업에 권력이 집중되어 책임 문제가 발생할 수 있습니다.
- AI 시스템의 공정성, 책임성, 투명성을 보장하는 것은 윤리적 배포를 위해 필수적입니다.
"""

# 다양한 압축 기법을 사용하여 평가 실행
# 압축 방식:
# - "selective": 중요 정보는 유지하고 덜 중요한 내용은 생략
# - "summary": 질의에 관련된 내용을 간결하게 요약
# - "extraction": 관련 문장을 문서에서 그대로 추출
results = evaluate_compression(  
    pdf_path=pdf_path,  
    query=query,  
    reference_answer=reference_answer,  
    compression_types=["selective", "summary", "extraction"]  
)


*** 문맥 압축 방식 평가 시작 ***
질문: 의사 결정에 AI를 사용하는 것과 관련된 윤리적 우려는 무엇인가요?

***표준 RAG 실행***
질문: 의사 결정에 AI를 사용하는 것과 관련된 윤리적 우려는 무엇인가요?
PDF에서 텍스트를 추출합니다...
텍스트를 청크 단위로 분할합니다...
21개의 청크가 생성되었습니다.
청크 임베딩을 생성합니다...
벡터 저장소를 초기화합니다...
21개의 청크가 벡터 저장소에 추가되었습니다.
상위 10개의 청크 검색 중...
응답 생성 중...

***최종 응답***
AI를 의사 결정에 사용하는 것과 관련된 윤리적 우려는 다음과 같습니다:

1. **편견과 공정성**: AI 시스템은 데이터에 존재하는 편견을 계승하고 증폭시켜 불공정하거나 차별적인 결과를 초래할 수 있습니다. 따라서 AI 시스템의 공정성을 보장하고 편견을 완화하는 것이 중요한 과제입니다.

2. **투명성 및 설명 가능성**: 많은 AI 시스템, 특히 딥러닝 모델은 '블랙박스'와 같아서 의사 결정 과정이 이해하기 어렵습니다. 투명성과 설명 가능성을 높이는 것은 신뢰와 책임감을 구축하는 데 매우 중요합니다.

3. **개인정보 보호 및 보안**: AI 시스템은 대량의 데이터에 의존하는 경우가 많기 때문에 개인정보 보호와 데이터 보안에 대한 우려가 제기됩니다. 민감한 정보를 보호하고 책임감 있는 데이터 처리를 보장하는 것이 필수적입니다.

4. **자율성 및 제어**: AI 시스템이 더욱 자율화됨에 따라 통제, 책임, 의도하지 않은 결과의 발생 가능성에 대한 의문이 제기되고 있습니다. AI 개발 및 배포를 위한 명확한 가이드라인과 윤리적 프레임워크를 수립하는 것이 중요합니다.

5. **AI의 무기화**: 자율 무기 시스템에 AI를 사용할 경우 심각한 윤리적, 보안적 우려가 제기될 수 있습니다. AI 기반 무기와 관련된 위험을 해결하기 위해 국제적인 논의와 규제가 필요합니다.

이러한 윤리적 우려들은 AI의 의사 결정 과정에서 신뢰성과 책임성을 확보하기 위

## Visualizing Compression Results

In [17]:
def visualize_compression_results(evaluation_results):
    """
    다양한 압축 방식의 결과를 시각적으로 비교 출력합니다.

    Args:
        evaluation_results (Dict): evaluate_compression 함수의 실행 결과.
    """
    # 질의 및 standard 방식의 청크 가져오기
    query = evaluation_results["query"]
    standard_chunks = evaluation_results["standard_result"]["chunks"]

    print(f"\n질문:\n{query}")
    print("\n" + "="*80 + "\n")

    # 비교용으로 standard 방식의 첫 번째 청크 선택
    original_chunk = standard_chunks[0]

    # 압축 방식별 비교 시각화
    for comp_type in evaluation_results["compression_results"].keys():
        compressed_chunks = evaluation_results["compression_results"][comp_type]["compressed_chunks"]
        compression_ratios = evaluation_results["compression_results"][comp_type]["compression_ratios"]

        compressed_chunk = compressed_chunks[0]
        compression_ratio = compression_ratios[0]

        print(f"\n=== {comp_type.upper()} COMPRESSION EXAMPLE ===\n")

        # 원본 청크 출력
        print("원본 청크:")
        print("-" * 40)
        if len(original_chunk) > 800:
            print(original_chunk[:800] + "... [중략]")
        else:
            print(original_chunk)
        print("-" * 40)
        print(f"문자 수: {len(original_chunk)}\n")

        # 압축 청크 출력
        print("압축된 청크:")
        print("-" * 40)
        print(compressed_chunk)
        print("-" * 40)
        print(f"문자 수: {len(compressed_chunk)}")
        print(f"해당 청크 압축률: {compression_ratio:.2f}%\n")

        # 전체 평균 압축률 및 길이 감소 정보
        avg_ratio = sum(compression_ratios) / len(compression_ratios)
        print(f"전체 평균 압축률: {avg_ratio:.2f}%")
        print(f"총 문맥 길이 감소율: {evaluation_results['metrics'][comp_type]['avg_compression_ratio']}")
        print("-" * 80)

    # 압축 방식별 요약 통계 표 출력
    print("\n*** 압축 방식 요약 통계 ***\n")
    print(f"{'방식':<15} {'평균 압축률':<20} {'압축 후 길이':<18} {'원본 길이':<15}")
    print("-" * 70)
    for comp_type, metrics in evaluation_results["metrics"].items():
        print(f"{comp_type:<15} {metrics['avg_compression_ratio']:<20} {metrics['total_context_length']:<18} {metrics['original_context_length']:<15}")

In [18]:
# 결과 보기
visualize_compression_results(results)


질문:
의사 결정에 AI를 사용하는 것과 관련된 윤리적 우려는 무엇인가요?



=== SELECTIVE COMPRESSION EXAMPLE ===

원본 청크:
----------------------------------------
 등에 
사용됩니다. AI 알고리즘은 대규모 데이터 세트를 분석하여 패턴을 파악하고, 시장 동향을 
예측하고, 금융 프로세스를 자동화할 수 있습니다. 
교통편 
AI는 자율주행차, 교통 최적화 시스템, 물류 관리의 발전으로 교통 분야에 혁신을 일으키고 
있습니다. 자율 주행 차량은 AI를 사용하여 주변 환경을 인식하고, 주행 결정을 내리고, 
안전하게 주행합니다. 
리테일 
리테일 업계에서는 개인화된 추천, 재고 관리, 고객 서비스 챗봇, 공급망 최적화를 위해 
AI를 사용합니다. AI 기반 시스템은 고객 데이터를 분석하여 수요를 예측하고, 제안을 
개인화하며, 쇼핑 경험을 개선할 수 있습니다. 
제조 
AI는 제조업에서 예측 유지보수, 품질 관리, 프로세스 최적화, 로봇 공학에 사용됩니다. AI 
기반 시스템은 장비를 모니터링하고, 이상 징후를 감지하고, 작업을 자동화하여 효율성을 
높이고 비용을 절감할 수 있습니다. 
교육 
AI는 개인화된 학습 플랫폼, 자동화된 채점 시스템, 가상 튜터를 통해 교육을 향상시키고 
있습니다. AI 기반 도구는 학생 개개인의 필요에 맞게 조정하고 피드백을 제공하며 맞춤형 
학습 환경을 조성할 수 있습니다. 
엔터테인먼트 
엔터테인먼트 업계에서는 콘텐츠 추천, 게임 개발, 가상 현실 경험에 AI를 사용합니다. AI 
알고리즘은 사용자 선호도를 분석하여 영화, 음악, 게임을 추천함으로써 사용자 참여도를 
높입니다. 
사이버 보안 
AI는 사이버 보안에서 위협을 탐지 및 대응하고, 네트워크 트래픽을 분석하고, 취약점을 
식별하는 데 사용됩니다. AI 기반 시스템은 보안 작업을 자... [중략]
----------------------------------------
문자 수: 1000

압축된 청